In [17]:
import os
import numpy as np
import pandas as pd
import torch
            
            

In [18]:
from utilities import sliding_windows


inp = np.load('./datasets/OpportunityUCIDataset/loco_2_mask.npy', allow_pickle=True)
inp.item()['inputs'].shape
inputs, labels = inp.item()['inputs'], inp.item()['labels']
sw = sliding_windows(100, 50)
segmented_samples, segmented_labels = sw(torch.tensor(inputs), torch.tensor(labels))
print(segmented_samples.shape, segmented_labels.shape)

torch.Size([8145, 100, 220]) torch.Size([8145, 100])


In [19]:
# custom Dataset:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data, label, transform=None):
        self.data = data
        self.label = label
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        if self.transform:
            return self.transform(self.data[idx]), self.label[idx]
        return self.data[idx], self.label[idx]

In [20]:
from utilities import seed
from utils_loader import MultiEpochsDataLoader
from sklearn.discriminant_analysis import StandardScaler
import torch

class StandardTransform(torch.nn.Module):
    def __init__(self, scaler='standard'):
        super(StandardTransform, self).__init__()
        if scaler == 'standard':
            self.scaler = StandardScaler()
        else:
            raise NotImplementedError('Only standard scaler is implemented')
        
    def __call__(self, data):
        data = self.scaler.transform(data)
        return torch.tensor(data)
    
    def fit(self, data):
        n_samples, n_time_steps, n_features = data.shape
        data_reshaped = data.reshape(-1, n_features)  # The shape becomes (n_samples * n_time_steps, n_features)
        self.scaler.fit(data_reshaped)
        print('Fitted with mean: {}, and std: {}'.format(self.scaler.mean_, np.sqrt(self.scaler.var_)))
        return self
    

seed(73054772)
# create train, val, test dataset:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(segmented_samples, segmented_labels, test_size=0.3, random_state=42, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42, shuffle=True)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

d, l = next(iter(train_loader))

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import ModuleList

from torch.nn.modules.transformer import TransformerEncoder, TransformerEncoderLayer


class SimpleSegmenterMaskTransformer(nn.Module):
    def __init__(self, in_channels=6, num_layers=2, num_heads=4, embed_dims=256, **kwargs):
        super(SimpleSegmenterMaskTransformer, self).__init__(**kwargs)

        # Fixed parameters for simplicity
        mlp_ratio = 4
        # norm_cfg = dict(type='LN')
        # act_cfg = dict(type='GELU')
        self.init_std = 0.02
        self.num_classes = 5
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            layer = nn.TransformerEncoderLayer(
                d_model=embed_dims,
                nhead=num_heads,
                dim_feedforward=mlp_ratio * embed_dims,
                dropout=0.5,
                activation=F.gelu,
                layer_norm_eps=1e-05,
                batch_first=True,
                norm_first=False,
                bias=True
            )
            self.layers.append(layer)

        self.dec_proj = nn.Linear(in_channels, embed_dims)
        self.cls_emb = nn.Parameter(torch.randn(1, self.num_classes, embed_dims))
        self.patch_proj = nn.Linear(embed_dims, embed_dims, bias=False)
        self.classes_proj = nn.Linear(embed_dims, embed_dims, bias=False)
        self.decoder_norm = nn.LayerNorm(embed_dims)
        self.mask_norm = nn.LayerNorm(self.num_classes)
        self.init_weights()
    def init_weights(self):
        nn.init.trunc_normal_(self.cls_emb, std=self.init_std)
        nn.init.trunc_normal_(self.patch_proj.weight, std=self.init_std)
        nn.init.trunc_normal_(self.classes_proj.weight, std=self.init_std)
        # Initialize weights for Transformer layers
        for layer in self.layers:
            for param in layer.parameters():
                if param.dim() > 1:
                    nn.init.xavier_uniform_(param)

    def forward(self, inputs):
        x = inputs.permute(0, 2, 1) # b h c
        b, c, h = x.shape
        x = x.view(b, c, -1).permute(0, 2, 1)

        x = self.dec_proj(x)
        cls_emb = self.cls_emb.expand(x.size(0), -1, -1)
        x = torch.cat((x, cls_emb), 1)
        for layer in self.layers: 
            x = layer(x)
        x = self.decoder_norm(x)

        patches = self.patch_proj(x[:, :-self.num_classes]) # shape 128, 75, 64
        cls_seg_feat = self.classes_proj(x[:, -self.num_classes:])
        # cls_seg_feat = nn.functional.dropout(cls_seg_feat, p=0.5, training=self.training)
        patches = F.normalize(patches, dim=2, p=2)
        cls_seg_feat = F.normalize(cls_seg_feat, dim=2, p=2)
        # print(patches.shape)
        masks = patches @ cls_seg_feat.transpose(1, 2)
        masks = self.mask_norm(masks).contiguous().view(b, h, -1)

        return masks

    def forward_pred(self, inputs):
        masks = self.forward(inputs)
        masks = masks.permute(0, 2, 1)
        probabilities = F.softmax(masks, dim=1)
        pred = torch.argmax(probabilities, dim=1)
        return pred


In [22]:
def mean_iou_time_series(pred, gt, num_classes=7):
    """
    Calculate the Mean Intersection over Union (mIoU) for time series data.
    
    Parameters:
        pred (torch.Tensor): Predictions, assumed to be one-hot encoded.
        gt (torch.Tensor): Ground truth labels, assumed to be one-hot encoded.
        num_classes (int): Number of classes in the segmentation task.

    Returns:
        float: Mean IoU for the batch.
    """
    # Initialize variables to store IoU for each class
    iou_list = []

    # Loop through each class
    for cls in range(num_classes):
        pred_inds = pred == cls
        gt_inds = gt == cls

        # Calculate Intersection and Union
        # print(pred_inds.shape, gt_inds.shape)
        intersection = torch.sum(pred_inds & gt_inds, dim=1).float()
        union = torch.sum(pred_inds | gt_inds, dim=1).float()

        # Calculate IoU. Avoid division by zero by adding a small epsilon.
        iou = intersection / (union + 1e-8)

        # Append the mean IoU for this class
        iou_list.append(torch.mean(iou))

    # Calculate the mean IoU across all classes
    mean_iou = torch.mean(torch.stack(iou_list))

    return mean_iou.item()


def mean_iou(preds, labels, num_classes):
    # Flatten the predictions and labels, this comes from the argmax of the one-hot vectors
    preds = preds.view(-1)
    labels = labels.view(-1)

    # Create confusion matrix
    confusion_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)
    for i in range(num_classes):
        for j in range(num_classes):
            confusion_matrix[i, j] = torch.sum((preds == i) & (labels == j))

    # Calculate IoU for each class
    ious = []
    for i in range(num_classes):
        true_positive = confusion_matrix[i, i]
        false_positive = confusion_matrix[i, :].sum() - true_positive
        false_negative = confusion_matrix[:, i].sum() - true_positive

        # Avoid division by zero
        union = true_positive + false_positive + false_negative
        if union == 0:
            ious.append(float('nan'))  # No predictions and no labels for this class
        else:
            ious.append(true_positive.float() / union.float())

    # Calculate mean IoU
    ious = torch.tensor(ious)
    mean_iou = torch.nanmean(ious)  # Mean over all classes, ignoring NaNs
    return mean_iou


In [23]:
# Create an instance of the CNN model
from livelossplot import PlotLosses
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from utilities import get_device
num_epochs = 100
device = get_device()
def c_loss(logits):
    probabilities = F.softmax(logits, dim=1)
    one_hot_predictions = torch.argmax(probabilities, dim=1)
    product_prob = torch.prod(probabilities, dim=1)
    changes = torch.abs(one_hot_predictions[:, 1:]*product_prob[:, 1:] - one_hot_predictions[:, :-1]*product_prob[:, :-1])
    penalty = changes.sum(dim=1).float().mean()
    return penalty + 1e-4
def custom_loss(logits, targets, penalty_weight=0.1):
    """
    Custom loss function that penalizes frequent changes in the prediction sequence.
    
    Args:
    - logits: Tensor of shape [N, C, L], where N is batch size, C is number of classes, and L is sequence length.
    - targets: Tensor of shape [N, L] containing class indices for each element in the sequence.
    - penalty_weight: Weight of the penalty term.

    Returns:
    - Total loss with the penalty.
    """

    # Standard cross-entropy loss
    probabilities = F.softmax(logits, dim=1)
    ce_loss = F.cross_entropy(probabilities, targets)

    penalty = c_loss(logits)
    total_loss = ce_loss #penalty * penalty_weight

    return total_loss, ce_loss.item(), penalty.item()

def plotASample(data, label):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)  # Add a new subplot to the figure
    ax.plot(data, label='Data')  # Plot the data
    ax.plot(label, label='Label')  # Plot the label
    ax.legend()  # Add a legend
    fig.canvas.draw()  # Redraw the figure
    plt.plot()


model = SimpleSegmenterMaskTransformer(in_channels=220).to(device)

# Define the loss function and optimizer
criterion =custom_loss #nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.00)
liveloss = PlotLosses()

# Iterate over the training data
logs = {}
# change the plt size:
best_loss = math.inf
counter_i = 0
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        images = images.float().to(device)
        labels = labels.to(device)
        outputs = model(images)
        outputs = outputs.permute(0, 2, 1)
        loss, tra_ce, tra_pen = criterion(outputs, labels.long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # train loss and accuracy check:
        if counter_i % 1000 == 0 and counter_i != 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            _, predicted = torch.max(outputs.data, 1)
            # total = labels.size(0)
            # correct = (predicted == labels).sum().item()
            # print('Accuracy: {:.2f}%'.format(correct / total * 100))
            logs['loss'] = loss.item()
            logs['ce'] = tra_ce
            logs['pen'] = tra_pen
            # logs['accuracy'] = correct / total * 100
    
            # validation loss and accuracy check:
            correct = 0
            total = 0
            for images, labels in val_loader:
                images = images.float().to(device)
                labels = labels.to(device)
                outputs = model(images)
                outputs = outputs.permute(0, 2, 1) #view(images.shape[0], 2, -1)
                loss, val_ce, val_pen = criterion(outputs, labels.long())
            logs['val_loss'] = loss.item()
            logs['val_ce'] = val_ce
            logs['val_pen'] = val_pen
            logs['miou'] = mean_iou(model.forward_pred(images), labels.long(),num_classes=5)
            if best_loss > loss.item():
                best_loss = loss.item()
                torch.save(model.state_dict(), './saved_model/best_transformer_oppo.pth')
            pred = torch.argmax(torch.softmax(outputs[0], dim=0), dim=0).detach().cpu().numpy()
            plotASample(images[0].detach().cpu().numpy()[1:7], pred)
            liveloss.update(logs)
            liveloss.send()
            # logs['val_accuracy'] = correct / total * 100
        counter_i += 1


CUDA is available.
